In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

### Read File

In [2]:
edges = []

with open('CitationPH/cit-HepPh.txt', 'r') as outf:
    edges = outf.readlines()

edges = edges[4:]
print(len(edges))

sample_edge = 100

421578


In [3]:
dates = []

with open('CitationPH/cit-HepPh-dates.txt', 'r') as outf:
    dates = outf.readlines()

dates = dates[1:]
print(len(dates))

data_date = {}

for date in dates:
    date = date.replace("\n", "")
    if not date.split()[0] in data_date:
        data_date[date.split()[0]] = {}
        data_date[date.split()[0]]['dates'] = []
        data_date[date.split()[0]]['dates'].append(date.split()[1])
    else:
        data_date[date.split()[0]]['dates'].append(date.split()[1])

38557


### Create Graph

In [4]:
graph =  nx.DiGraph()

for edge in edges[:sample_edge]:
    edge = edge.replace("\n", "")
    graph.add_edge(edge.split()[0], edge.split()[1])

In [5]:
print(len(graph.nodes))
print(len(graph.edges))

84
100


### Count in_degree and out_degree

In [6]:
nodes = list(graph.nodes)
data = {}
# data_list = []

node_no_out_d = set()
node_no_in_d = set()

for node in nodes:
    in_degree = graph.in_degree(node)
    if in_degree == 0:
        node_no_in_d.add(node)
    out_degree = graph.out_degree(node)
    if out_degree == 0:
        node_no_out_d.add(node)
    data[node] = [in_degree, out_degree, data_date[date.split()[0]]['dates'][-1]]
#     data_list.append((node, in_degree, out_degree))

print('no out degree ', len(node_no_out_d))
print('no in degree ', len(node_no_in_d))
    
pd_data = pd.DataFrame.from_dict(data, orient='index', columns=['in_degree', 'out_degree', 'dates'])
display(pd_data.head())
display(pd_data.describe())

# pd_data_list = pd.DataFrame.from_records(data_list, columns=['nodes', 'in_degree', 'out_degree'])
# display(pd_data_list)
# display(pd_data.describe())

no out degree  71
no in degree  2


,in_degree,out_degree,dates
9907233,0,11,2002-03-11
9301253,3,0,2002-03-11
9504304,3,0,2002-03-11
9505235,4,0,2002-03-11
9506257,1,1,2002-03-11


,in_degree,out_degree
count,84.000000,84.000000
mean,1.190476,1.190476
std,0.610672,3.276334
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,0.000000
max,4.000000,17.000000


### Draw Graph

In [9]:
import pygraphviz as pgv

out_graph = pgv.AGraph(directed=True)
in_graph = pgv.AGraph(directed=True)

for edge in edges[:sample_edge]:
    edge = edge.replace("\n", "")
    out_graph.add_edge(edge.split()[0], edge.split()[1])
    in_graph.add_edge(edge.split()[0], edge.split()[1])

# pg_graph.draw('graphcitation.png',prog="circo")

In [10]:
in_degree_max = float(pd_data.max()['in_degree'])
out_degree_max = float(pd_data.max()['out_degree'])

for node in data:
    in_node_graph = in_graph.get_node(node)
    out_node_graph = out_graph.get_node(node)
    in_degree = data[node][0]
    out_degree = data[node][1]
    out_node_graph.attr['height']="%s"%(out_degree/out_degree_max+0.5)
    out_node_graph.attr['width']="%s"%(out_degree/out_degree_max+0.5)
    in_node_graph.attr['heigth']="%s"%(in_degree/in_degree_max+0.5)
    in_node_graph.attr['width']="%s"%(in_degree/in_degree_max+0.5)
    
out_graph.draw('out_graphcitation.png',prog="circo")
in_graph.draw('in_graphcitation.png', prog="circo")